In [1]:
!nvidia-smi

Wed Feb  9 17:28:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:09:00.0  On |                  N/A |
| 59%   65C    P2   226W / 420W |  10750MiB / 24576MiB |     42%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torchaudio
import librosa
import time

from asteroid.metrics import get_metrics

from asteroid_filterbanks import STFTFB, Encoder, Decoder
from asteroid.masknn import TDConvNet , TDConvNetpp
from asteroid_filterbanks import make_enc_dec
from asteroid import ConvTasNet
from asteroid.data import LibriMix
from asteroid.losses import PITLossWrapper, pairwise_neg_sisdr
from asteroid.data import LibriMix
from asteroid.engine.system import System
from asteroid.losses import PITLossWrapper, pairwise_neg_sisdr
from asteroid import ConvTasNet

import matplotlib.pyplot as plt
import numpy as np
import IPython.display as ipd

import joblib
import os
import csv
import numpy as np
import IPython.display as ipd
import matplotlib.pyplot as plt
import librosa
import pandas as pd
import soundfile as sf

from torchsummary import summary
import torch.nn as nn
import torch
from torchvision import transforms, utils
import torchaudio

import torch.optim as optim
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
#device = 'cpu'
import torch.nn.functional as F
import random

def returnNormedVector_oneSam(A = None):
    
    n_channels = 1
    n_sam = A.shape[-1]
    AA = A.clone()
    t_zro = torch.zeros(n_channels, n_sam+1)
    t_zro[:,:n_sam] = AA
    AA = t_zro
    
    #AA = AA.view(A.size(0), -1)
    AA -= AA.min(1, keepdim=True)[0]
    AA /= AA.max(1, keepdim=True)[0]
    AA = AA-AA[0][n_sam]
    
    AA = AA.view(n_channels, n_sam+1)
    
    return AA[:,:n_sam]


cuda


In [2]:
# Model definition based on the design of model #1

from asteroid_filterbanks import make_enc_dec
from asteroid_filterbanks import STFTFB

class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        fb = STFTFB(n_filters=256, kernel_size=128, stride=64)
        self.enc = Encoder(fb)
        self.cond_enc = Encoder(fb)
     
        
        decoder_fb = STFTFB(n_filters=514, kernel_size=128, stride=64)
        self.dec = Decoder(decoder_fb)
        
        self.masker = TDConvNet(in_chan=516, 
                                n_src=1)
        self.m = nn.Linear(14, 64) 
        self.m2 = nn.Linear(64, 499)
        
        self.dropout = nn.Dropout(0.25)
        
        self.c1 = nn.Conv1d(in_channels=1, out_channels= 258, kernel_size = 3, stride=1,padding = 1,  padding_mode ='zeros')
        self.bn1 = nn.BatchNorm1d(258)
        
        self.c1d_1 = nn.Conv1d(in_channels=516, out_channels= 516, kernel_size = 3, stride=1,padding = 1,  padding_mode ='zeros')
        self.bn2 = nn.BatchNorm1d(516)
        
        
        # For the FC layers in the binary Classification
        self.co_fc_1 = nn.Conv1d(in_channels=516, out_channels= 1, kernel_size = 3, stride=1,padding = 1,  padding_mode ='zeros')
        
            
        # Number of input features is 516x499.
        self.fc1 = nn.Linear(499, 32) 
        self.fc2 = nn.Linear(32, 1)
        #self.fc3 = nn.Linear(32, 1) 
        
        #self.relu = nn.ReLU()
        self.dropout_2 = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)
        #self.softmax = nn.LogSoftmax(dim=1)
        self.sigmoid = nn.Sigmoid()
        

    def forward(self, wav, cond_wav):
        
        #print("Shapes of the input mix and cond:")
        #print(wav.shape)
        #print(cond_wav.shape)
        tf_rep = self.enc(wav)
        #print("00_tf_rep",tf_rep.shape)

        m_out = self.m(cond_wav)
        m_out = F.relu(m_out) # need dropout
        m_out = self.dropout(m_out)
        #print("01_m_out", m_out.shape)
        
        m2_out = self.m2(m_out)
        m2_out = F.relu(m2_out) # need dropout, changed to sigmoid from relu
        m2_out = self.dropout(m2_out)
        #print("01_m2_out", m2_out.shape)
        
        c1_out = self.c1(m2_out)
        c1_out = F.relu(self.bn1(c1_out))
        #print("01_c1_out", c1_out.shape)
                
        concat_in = torch.cat((tf_rep, c1_out),dim=1)
        #print("03_concat_in:",concat_in.shape)
        
        c1d_1_out = self.c1d_1(concat_in)
        c1d_1_out = F.relu((c1d_1_out)) # removed <self.bn2> from here
        #print("04_c1d_1_out:",c1d_1_out.shape)

        masks = self.masker(c1d_1_out)
        #print("04_masks Shape: ",masks.shape)
        #print("Concat the above 04s")
        
        con_usq = c1d_1_out.unsqueeze(1) * masks
        #print("Post mask multiplication, con_usq:", con_usq.shape)
        
        # Add the pres_abs binary detector here
        co_fc_1_out = self.co_fc_1(con_usq.squeeze(1))
        #print("05_co_fc_1_out",co_fc_1_out.shape)
        
        x = co_fc_1_out.view(-1, 499)
        x = self.dropout_2(F.relu(self.fc1(x)))
        x = self.sigmoid(self.fc2(x)) # Pres/abs
        
        wavs_out = self.dec(con_usq) # output sep waveform
        
        return wavs_out, x

print(device)
# Define and forward 
cond_conv_tasnet = Model()
cond_conv_tasnet.to(device)
wav_out, pres_abs = cond_conv_tasnet(wav = torch.randn(10, 1, 32000).to(device), cond_wav = torch.randn(10, 1, 14).to(device))   
print(wav_out.shape, pres_abs.shape)

cuda
torch.Size([10, 1, 32000]) torch.Size([10, 1])


In [3]:
def count_parameters(cond_conv_tasnet):
    return sum(p.numel() for p in cond_conv_tasnet.parameters() if p.requires_grad)

n = count_parameters(cond_conv_tasnet)
print("Number of parameters: %s" % n)

Number of parameters: 5822246


In [5]:
PATH = '../saved_model/ohot_Nigens-5M_EP_38.pt'
cond_conv_tasnet.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [6]:
csv_file = '/home/shree//21/cond_sep/icassp2022/from_server/nigen/icassp22_exp/processed_data/csv/val_combinations.csv'
'''wav_dir = '/home/shree//21/cond_sep/icassp2022/from_server/nigen/icassp22_exp/processed_data/val/'
mix_dir = '/home/shree//21/cond_sep/icassp2022/from_server/nigen/icassp22_exp/processed_data/processed_mixed_data/val/'
'''

"wav_dir = '/home/shree//21/cond_sep/icassp2022/from_server/nigen/icassp22_exp/processed_data/val/'\nmix_dir = '/home/shree//21/cond_sep/icassp2022/from_server/nigen/icassp22_exp/processed_data/processed_mixed_data/val/'\n"

In [7]:
data = pd.read_csv(csv_file)
data_pth = '/home/shree/21/cond_sep/icassp2022/from_server/nigen/icassp22_exp/processed_data/val/'
snr_d = '/media/first/icassp_res/2022_Feb09_simple/snr0'


os.makedirs(snr_d, exist_ok = True)

spec = ['alarm','baby','crash','dog','engine','femaleScream',
         'femaleSpeech','fire','footsteps','knock','maleScream',
         'maleSpeech','phone','piano']

cond_conv_tasnet.eval()

for index, row in data.iterrows():
    #print(row['wav1_target'], row['wav2'], row['look_for'],row['pres_abs'])
    
    if(row['pres_abs'] == 1):
        
        save_to = os.path.join(snr_d, str(index))
        os.makedirs(save_to, exist_ok=True)
    
        sam_1, sr = torchaudio.load(os.path.join(data_pth, (row['look_for'] + '/' + row['wav1_target'])))
        print(sam_1.size())

        sam_2, sr = torchaudio.load(os.path.join(data_pth, (row['wav2'].split('_')[3]+ '/' + row['wav2'])))
        print(sam_2.size())
        
        #mix_ = sam_1 + (sam_2 * 0.50)
        mix_ = sam_1 + (sam_2) #<--------------- SNR is changed here
        #mix_ = sam_1 + (sam_2 * 0.25)
        
        mix= returnNormedVector_oneSam(mix_)
        
        cond_look_for = row['look_for']

        spec.index(cond_look_for) + 1
        cond = np.zeros(14) # <<-- No. of Classes 14 in case of NIGEN
        cond[spec.index(cond_look_for)] = 1
        cond_ = torch.from_numpy(np.float32(cond)) # This is what to look for, hence a conditional
        cond_ = cond_.unsqueeze(0)
        
        cond_look_for = row['look_for']
        
        print(">>",row['wav1_target'],row['wav2'])
        
        with torch.no_grad():
            output, x_out = cond_conv_tasnet(wav = torch.unsqueeze(mix, 0).to(device), cond_wav = torch.unsqueeze(cond_, 0).to(device))
        
        print(">>>")
        est = output.cpu().numpy().flatten()/400
        clean = sam_1.cpu().numpy().flatten()
        mix= mix.cpu().numpy().flatten()
        print(">>>>>")
        
        #metrics_dict = get_metrics(mix, clean, est, sample_rate=16000,
        #                             metrics_list='all')
        #print(metrics_dict)
        
        sf.write(os.path.join(save_to, 'input_mixture.wav'),data = mix,   samplerate = 16000)
        sf.write(os.path.join(save_to, 'pred_separation.wav'),data = est,   samplerate = 16000)
        sf.write(os.path.join(save_to, 'cleanFile.wav'),data = clean,   samplerate = 16000)
        joblib.dump([x_out, 1],os.path.join(save_to, 'detection.pkl'))

FileNotFoundError: [Errno 2] No such file or directory: '/home/shree//21/cond_sep/icassp2022/from_server/nigen/icassp22_exp/processed_data/csv/val_combinations.csv'

In [8]:
# counting absesces detecion accuracy

abs_det = []
data_pth = '/home/shree//21/cond_sep/icassp2022/from_server/nigen/icassp22_exp/processed_data/val/'

for index, row in data.iterrows():
    #print(row['wav1_target'], row['wav2'], row['look_for'],row['pres_abs'])
    
    if(row['pres_abs'] == 0):
        
        sam_1, sr = torchaudio.load(os.path.join(data_pth, (row['wav1_target'].split('_')[3] + '/' + row['wav1_target'])))
        print(sam_1.size())

        sam_2, sr = torchaudio.load(os.path.join(data_pth, (row['wav2'].split('_')[3]+ '/' + row['wav2'])))
        print(sam_2.size())
        
        #mix_ = sam_1 + (sam_2 * 0.50)
        mix_ = sam_1 + (sam_2) #<--------------- SNR is changed here
        #mix_ = sam_1 + (sam_2 * 0.25)
        
        mix= returnNormedVector_oneSam(mix_)
        
        cond_look_for = row['look_for']

        spec.index(cond_look_for) + 1
        cond = np.zeros(14) # <<-- No. of Classes 14 in case of NIGEN
        cond[spec.index(cond_look_for)] = 1
        cond_ = torch.from_numpy(np.float32(cond)) # This is what to look for, hence a conditional
        cond_ = cond_.unsqueeze(0)
        
        cond_look_for = row['look_for']
        
        print(">>",row['wav1_target'],row['wav2'])
        
        with torch.no_grad():
            output, x_out = cond_conv_tasnet(wav = torch.unsqueeze(mix, 0).to(device), cond_wav = torch.unsqueeze(cond_, 0).to(device))
        
        abs_det.append([x_out.cpu().numpy().flatten(), 0])
        
        

torch.Size([1, 32000])
torch.Size([1, 32000])
>> 13_32000_2sec_maleSpeech_norm.wav 83_32000_2sec_maleScream_norm.wav
torch.Size([1, 32000])
torch.Size([1, 32000])
>> 16_32000_2sec_maleSpeech_norm.wav 204_32000_2sec_phone_norm.wav
torch.Size([1, 32000])
torch.Size([1, 32000])
>> 21_32000_2sec_maleSpeech_norm.wav 392_32000_2sec_footsteps_norm.wav
torch.Size([1, 32000])
torch.Size([1, 32000])
>> 8_32000_2sec_maleSpeech_norm.wav 222_32000_2sec_crash_norm.wav
torch.Size([1, 32000])
torch.Size([1, 32000])
>> 34_32000_2sec_maleSpeech_norm.wav 106_32000_2sec_piano_norm.wav
torch.Size([1, 32000])
torch.Size([1, 32000])
>> 11_32000_2sec_maleSpeech_norm.wav 19_32000_2sec_knock_norm.wav
torch.Size([1, 32000])
torch.Size([1, 32000])
>> 28_32000_2sec_maleSpeech_norm.wav 190_32000_2sec_piano_norm.wav
torch.Size([1, 32000])
torch.Size([1, 32000])
>> 24_32000_2sec_maleSpeech_norm.wav 11_32000_2sec_alarm_norm.wav
torch.Size([1, 32000])
torch.Size([1, 32000])
>> 27_32000_2sec_maleSpeech_norm.wav 88_32000

In [9]:
# abs accuracy:

tru_cnt = 0
tot_cnt = 0

for det_ in abs_det:
    if(det_[0] < 0.50): # great!
        tru_cnt+=1
    tot_cnt +=1

print(tru_cnt)
print(tot_cnt)

394
420


In [10]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torchaudio
import librosa
import time


from asteroid.metrics import get_metrics

from asteroid_filterbanks import STFTFB, Encoder, Decoder
from asteroid.masknn import TDConvNet , TDConvNetpp
from asteroid_filterbanks import make_enc_dec
from asteroid import ConvTasNet
from asteroid.data import LibriMix
from asteroid.losses import PITLossWrapper, pairwise_neg_sisdr
from asteroid.data import LibriMix
from asteroid.engine.system import System
from asteroid.losses import PITLossWrapper, pairwise_neg_sisdr
from asteroid import ConvTasNet

import matplotlib.pyplot as plt
import numpy as np
import IPython.display as ipd


import os
import csv
import numpy as np
import IPython.display as ipd
import matplotlib.pyplot as plt
import librosa
import pandas as pd
import soundfile as sf

from torchsummary import summary
import torch.nn as nn
import torch
from torchvision import transforms, utils
import torchaudio

import torch.optim as optim
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
device = 'cpu'
import torch.nn.functional as F
import random

def returnNormedVector_oneSam(A = None):
    
    n_channels = 1
    n_sam = A.shape[-1]
    AA = A.clone()
    t_zro = torch.zeros(n_channels, n_sam+1)
    t_zro[:,:n_sam] = AA
    AA = t_zro
    
    #AA = AA.view(A.size(0), -1)
    AA -= AA.min(1, keepdim=True)[0]
    AA /= AA.max(1, keepdim=True)[0]
    AA = AA-AA[0][n_sam]
    
    AA = AA.view(n_channels, n_sam+1)
    
    return AA[:,:n_sam]


cuda


In [11]:
def getMetrics(path_snr = None):
    clean ,sr = librosa.load(os.path.join(path_snr, 'cleanFile.wav'), sr = 16000)
    
    est, sr = librosa.load(os.path.join(path_snr, 'pred_separation.wav'), sr = 16000)
    mix, sr = librosa.load(os.path.join(path_snr, 'input_mixture.wav'), sr = 16000)

    metrics_dict = get_metrics(mix, clean, est, sample_rate=16000,
                             metrics_list= (['si_sdr', 'sdr','stoi']))
    return metrics_dict

snr_d # = '/media/first/icassp_res/2022_Feb02/snr12/'


'/media/first/icassp_res/2022_Feb09_simple/snr0'

In [12]:
avg_stoi = 0
avg_sdr = 0
avg_si_sdr = 0
det_cnt = 0
tot_det = 0

for en, dir_ in enumerate(os.listdir(snr_d)):
    
    try:
        metrics = getMetrics(os.path.join(snr_d,dir_))
        myList = ([[metrics['stoi'], metrics['si_sdr'], metrics['sdr']]])
    
        avg_stoi += metrics['stoi']
        avg_sdr += metrics['sdr']
        avg_si_sdr += metrics['si_sdr']
        
        save_to = (os.path.join(snr_d,dir_))
        detection = joblib.load(os.path.join(save_to, 'detection.pkl'))
        
        thresh = 0.50
        if(detection[0]>thresh):
            det_cnt+=1
        
        tot_det +=1
        
    except:
        print(en,".")
        print("Errror here:",os.path.join(snr_d,dir_))
        print("But moving On")
    
    

avg_sdr = avg_sdr/len(os.listdir(snr_d))
avg_si_sdr = avg_si_sdr/len(os.listdir(snr_d))
avg_stoi = avg_stoi/len(os.listdir(snr_d))
    
'''
    filename = os.path.join('./metrics_csv/', ('metrics_SNR_' + snr_pth.split('SNR_')[-1]+'.csv'))
    with open(filename, 'a') as csvfile: 
        csvwriter = csv.writer(csvfile) 
        csvwriter.writerows(myList)'''
print("\n")
print("\n")
print("\n")
print("*----------------------------*")
print("avg_sdr",avg_sdr)
print("avg_si_sdr",avg_si_sdr)
print("avg_stoi",avg_stoi)
print("\n")
print("*----------------------------*")
print(det_cnt)
tot_det

/home/shree/anaconda3/envs/pytorch/lib/python3.9/site-packages/pystoi/stoi.py:66: RuntimeWarning: Not enough STFT frames to compute intermediate intelligibility measure after removing silent frames. Returning 1e-5. Please check you wav files
  warnings.warn('Not enough STFT frames to compute intermediate '


184 .
Errror here: /media/first/icassp_res/2022_Feb09_simple/snr0/491
But moving On
234 .
Errror here: /media/first/icassp_res/2022_Feb09_simple/snr0/211
But moving On
376 .
Errror here: /media/first/icassp_res/2022_Feb09_simple/snr0/351
But moving On
473 .
Errror here: /media/first/icassp_res/2022_Feb09_simple/snr0/71
But moving On






*----------------------------*
avg_sdr 12.311841716254458
avg_si_sdr 10.810616855381522
avg_stoi 0.6717133416863185


*----------------------------*
466


556

In [13]:

print("avg_sdr",avg_sdr)
print("avg_si_sdr",avg_si_sdr)
print("avg_stoi",avg_stoi)

avg_sdr 12.311841716254458
avg_si_sdr 10.810616855381522
avg_stoi 0.6717133416863185


In [14]:
print(det_cnt)
tot_det

466


556